# Imports

In [1]:
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import pandas as pd
from pandas.errors import EmptyDataError
import statistics
from tkinter import *
from tkinter import ttk, filedialog

# Functions

Data Tab Functions

In [2]:
# FILE OPENING


def csv_open(file_label):  # imports files with a dialogue box

    ## file dialogue

    import_file = filedialog.askopenfilename(
        initialdir="C:/", title="Open A CSV File", filetypes=(("CSV files", "*.csv"),)
    )  # opens file dialogue box

    ## data loading

    try:
        import_data = pd.read_csv(import_file)  # creates temp dataframe for csv data

        if file_label == airport_label:  # matches parameter to file label
            global airport_data  # makes data variable available globally
            airport_data = import_data  # copies temp data to global dataframe
        elif file_label == frequency_label:
            global frequency_data
            frequency_data = import_data
        elif file_label == runway_label:
            global runway_data
            runway_data = import_data
            clean_button.config(state=NORMAL)  # activates the cleaning button
        else:
            set_feedback("error", "An unknown error occured: please try again")

        ## success feedback

        set_feedback(
            "success", "File data loaded: load three files to proceed to data cleaning"
        )  # label confirms successful file load
        file_label.config(
            text=import_file[import_file.rfind("/") + 1 :] + " loaded",
            foreground="black",
        )  # sets file label to match the imported file name

    ## error handling

    except FileNotFoundError:
        set_feedback("error", "File not loaded: please select a file")
    except EmptyDataError:
        set_feedback("error", "File was empty or corrupted: please select another file")
    except:
        set_feedback("error", "An unknown error occured: please try again")


# DATA HANDLING


def clean_data():  # cleans data according to client brief

    ## function calls

    try:
        clean_airport()  # calls three helper functions to clean each individual file
        clean_frequency()
        clean_runway()

        ## feedback

        set_feedback(
            "success",
            "Data cleaned successfully: duplicate, irrelevant, and missing data removed",
        )

        merge_button.config(state=NORMAL)  # activates the merge button
        clean_button.config(state=DISABLED)  # deactivates the clean button

        airport_label.config(
            text="Cleaned", foreground="black",
        )
        frequency_label.config(
            text="Cleaned", foreground="black",
        )
        runway_label.config(
            text="Cleaned", foreground="black",
        )
        cleaned_label.config(text="Reload files to clean again")

    ## error handling
    except IndexError:
        set_feedback(
            "error", "Cleaning unsuccessful: reload the raw files and try again"
        )
    except NameError:
        set_feedback(
            "error", "Cleaning unsuccessful: reload the raw files and try again"
        )
    except AttributeError:
        set_feedback(
            "error", "Cleaning unsuccessful: reload the raw files and try again"
        )
    except KeyError:
        set_feedback(
            "error", "Cleaning unsuccessful: reload the raw files and try again"
        )


def merge_data():  # merges files on two columns

    ## global values

    global clean_merged  # allows variable to accessed by other functions
    clean_merged = pd.merge(
        cleaned_airport,
        cleaned_frequency,
        how="inner",
        on=["airport_ref", "airport_ident"],
    )  # merges files on inner join of two columns

    # display data

    display_file(clean_merged)  # updates the treeview

    ## feedback

    set_feedback(
        "success",
        "Data merged successfully: raw data and graphs without outliers displayed",
    )

    backup_button.config(state=NORMAL)  # activates the backup button
    merge_button.config(state=DISABLED)  # deactivates the merge button
    restore_button.config(state=DISABLED)  # deactivates the load button

    merged_label.config(text="Merged data displayed")
    restore_label.config(text="")  # clears the restore file displayed message


# JSON BACKUPS


def backup_json():  # saves cleaned and merged data to JSON file

    ## save

    clean_merged.to_json(r"backup.json", orient="records")  # saves to backup.json file

    ## feedback

    set_feedback("success", "Backup saved successfully: backup.json file created")

    saved_label.config(text="backup.json created")  # temporary success message
    saved_label.after(4000, lambda: saved_label.config(text=""))


def restore_json():  # loads clean and merged data from JSON file

    ## load

    try:
        global clean_merged  # accesses global variable
        clean_merged = pd.read_json(r"backup.json")  # loads backup.json file
        display_file(clean_merged)  # updates the treeview

        ## feedback

        set_feedback(
            "success",
            "Backup loaded successfully: raw data and graphs without outliers displayed",
        )

        airport_button.config(state=DISABLED)
        frequency_button.config(state=DISABLED)
        runway_button.config(state=DISABLED)
        restore_button.config(state=DISABLED)

        airport_label.config(text="")
        frequency_label.config(text="")
        runway_label.config(text="")
        restore_label.config(text="backup.json displayed")
        merged_label.config(text="")  # clears the merged file displayed message

    ## error handling

    except ValueError:
        set_feedback(
            "error",
            "Backup file has been moved, renamed, or does not exist: reload data and save again",
        )

Data Tab Helper Functions

In [3]:
## CLEANING


def clean_airport():  # cleans the airport file as per client brief

    ## global values

    global cleaned_airport  # allows variable to accessed by other functions
    cleaned_airport = airport_data

    ## remove duplicates

    cleaned_airport.drop_duplicates(
        subset=["id"], keep="last", inplace=True
    )  # removes duplicate ids

    ## remove irrelevant

    cleaned_airport = cleaned_airport[
        (cleaned_airport.iso_region == "GB-ENG")
        | (cleaned_airport.iso_region == "GB-NIR")
        | (cleaned_airport.iso_region == "GB-SCT")
        | (cleaned_airport.iso_region == "GB-WLS")
    ]  # removes any non-GB airports

    cleaned_airport = cleaned_airport[
        (cleaned_airport.type == "small_airport")
        | (cleaned_airport.type == "medium_airport")
        | (cleaned_airport.type == "large_airport")
    ]  # removes any airport not sml med or lrg

    ## fix structural errors

    cleaned_airport.rename(
        columns={"id": "airport_ref", "ident": "airport_ident"}, inplace=True
    )  # renames id column to match naming in the other two files

    cleaned_airport["small_airport"] = np.where(
        cleaned_airport["type"] == "small_airport", "S", ""
    )  # seperates sml airports into a binary column

    cleaned_airport["medium_airport"] = np.where(
        cleaned_airport["type"] == "medium_airport", "M", ""
    )  # seperates med airports into a binary column

    cleaned_airport["large_airport"] = np.where(
        cleaned_airport["type"] == "large_airport", "L", ""
    )  # seperates lrg airports into a binary column

    cleaned_airport.drop(
        cleaned_airport.columns[[2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17,]],
        axis=1,
        inplace=True,
    )  # drop columns with no value to client brief

    ## handle missing data

    cleaned_airport.dropna(
        subset=["airport_ref", "airport_ident", "iso_region"], inplace=True
    )  # drops missing values in key columns only

    ## sorting

    cleaned_airport = cleaned_airport.sort_values(
        by="airport_ref"
    )  # sorts data by ref column for neater treeview display


def clean_frequency():  # cleans the frequency file as per client brief

    ## global values

    global cleaned_frequency  # allows variable to accessed by other functions
    cleaned_frequency = frequency_data  # copies imported data to global dataframe

    ## remove duplicates

    cleaned_frequency.drop_duplicates(
        subset=["airport_ref", "type", "frequency_mhz"], keep="last", inplace=True
    )  # remove duplicate airports with same frequency and type

    ## remove irrelevant

    cleaned_frequency.drop(
        cleaned_frequency.columns[[0, 4]], axis=1, inplace=True
    )  # drop columns with no value to client brief

    ## handle missing data

    cleaned_frequency.dropna(
        subset=["airport_ref", "frequency_mhz"], inplace=True
    )  # drops missing values in key columns only


def clean_runway():  # cleans the runway file as per client brief

    ## global values

    global cleaned_runway  # makes data variable available globally
    cleaned_runway = runway_data  # copies imported data to global dataframe

    ## remove duplicates

    cleaned_runway.drop_duplicates(
        subset=["id", "airport_ref", "airport_ident"], keep="last", inplace=True,
    )  # remove duplicate runways with same id, ref, and ident

    ## remove irrelevant

    cleaned_runway.drop(
        cleaned_runway.columns[
            [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,]
        ],
        axis=1,
        inplace=True,
    )  # drop columns with no value to client brief

    ## handle missing data

    cleaned_runway.dropna(
        subset=["airport_ref"], inplace=True
    )  # drops rows with missing values in key columns only


## DATA DISPLAY


def display_file(data):  # displays data into treeview

    ## clears

    clear_tree()  # clears existing data

    ## populate

    data_tree["show"] = "headings"

    data_rows = data.to_numpy().tolist()
    for row in data_rows:  # iterates over data and puts into rows
        data_tree.insert("", "end", values=row)

    data_tree.pack()
    show_graphs()


def clear_tree():  # deletes entries to allow new data to be displayed

    ## clears

    data_tree.delete(*data_tree.get_children())  # deletes any current treeview

Averages and Visualisation Tab Functions

In [4]:
# GRAPHS


def show_graphs():

    ## data cleaning

    global graph_data
    graph_data = clear_outliers(clean_merged)  # clear frequency outliers from data set

    ## graph drawing

    airport_graph()
    frequency_graph()
    distribution_graph()
    correlation_graph()

Averages and Visualisation Tab Helper Functions

In [5]:
# CALCULATIONS


def clear_outliers(data):  # clears outliers from cleaned data

    ## data handling

    q_one = data["frequency_mhz"].quantile(0.25)  # calculates quartile one
    q_three = data["frequency_mhz"].quantile(0.75)  # calculates quartile three
    iqr = q_three - q_one  # calculates interquartile range
    lower_bound = q_one - 1.5 * iqr  # extends quartile one
    upper_bound = q_three + 1.5 * iqr  # extends quartile three

    data = data[
        (data.frequency_mhz > lower_bound) & (data.frequency_mhz < upper_bound)
    ]  # removes above and below threshold

    return data


def calculate_averages(data):

    ## calculations

    averages = []  # list to hold the mean, median, mode

    mean = averages.append(statistics.mean(data))
    median = averages.append(statistics.median(data))
    mode = averages.append(statistics.mode(data))

    return averages


# GRAPHS


def airport_graph():

    ## data handling

    data = graph_data.loc[
        graph_data["large_airport"] == "L", "frequency_mhz"
    ]  # copies global dataframe and filters only large airport data
    averages_data = calculate_averages(data)  # calculate averages

    ## graph drawing

    averages_left.clf()  # clears existing figures
    graph_plot = averages_left.add_subplot(111)  # creates graph subplot

    bins = [*range(105, 145, 1)]  # generate static bins for all histograms
    graph_plot.hist(data, bins, color="silver")  # creates the graph

    airport_canvas.draw()  # redraws the canvas to display changes
    airport_canvas.get_tk_widget().grid()

    ## graph design

    graph_plot.set_title("Large Airport Average Frequencies\n")  # sets the title
    graph_plot.set_xlabel("MHz")  # sets the x axis label
    graph_plot.set_ylabel("Usage")  # sets the y axis label
    graph_plot.set_axisbelow(True)  # puts grid behind drawn bars
    graph_plot.grid(True, alpha=0.2, ls="-")  # adds a faded grid
    graph_plot.margins(x=0, y=0.05663)
    graph_plot.set_ylim(0, 40)  # standardises the vertical scale across graphs

    # annotations and legend

    graph_plot.axvline(
        averages_data[0],
        color="#f64d2a",
        ls="-",
        label="Mean {:0.2f}".format(averages_data[0]),
    )  # draws and labels the mean line
    graph_plot.axvline(
        averages_data[1],
        color="#8abb21",
        ls="-",
        label="Median {:0.2f}".format(averages_data[1]),
    )  # draws and labels the median line
    graph_plot.axvline(
        averages_data[2],
        color="#2d95ec",
        ls="-",
        label="Mode {:0.2f}".format(averages_data[2]),
    )  # draws and labels the mode line

    graph_plot.legend(frameon=False)


def frequency_graph():

    ## data handling

    data = graph_data.loc[
        graph_data["frequency_mhz"] >= 100, "frequency_mhz"
    ]  # copies global dataframe and filters only frequencies above 100MHz
    averages_data = calculate_averages(data)  # calculate averages

    ## graph drawing

    averages_right.clf()  # clears existing figures
    graph_plot = averages_right.add_subplot(111)  # creates graph subplot

    bins = [*range(105, 145, 1)]  # generate static bins for all histograms
    graph_plot.hist(data, bins, color="silver")  # creates the graph

    frequency_canvas.draw()  # redraws the canvas to display changes
    frequency_canvas.get_tk_widget().grid()

    ## graph design

    graph_plot.set_title("Over 100 MHz Average Frequencies\n")  # sets the title
    graph_plot.set_xlabel("MHz")  # sets the x axis label
    graph_plot.set_ylabel("Usage")  # sets the y axis label
    graph_plot.set_axisbelow(True)  # puts grid behind drawn bars
    graph_plot.grid(True, alpha=0.2, ls="-")  # adds a faded grid
    graph_plot.margins(x=0, y=0.05663)
    graph_plot.set_ylim(0, 80)  # standardises the vertical scale across graphs

    # annotations and legend

    graph_plot.axvline(
        averages_data[0],
        color="#f64d2a",
        ls="-",
        label="Mean {:0.2f}".format(averages_data[0]),
    )  # draws and labels the mean line
    graph_plot.axvline(
        averages_data[1],
        color="#8abb21",
        ls="-",
        label="Median {:0.2f}".format(averages_data[1]),
    )  # draws and labels the median line
    graph_plot.axvline(
        averages_data[2],
        color="#2d95ec",
        ls="-",
        label="Mode {:0.2f}".format(averages_data[2]),
    )  # draws and labels the mode line

    graph_plot.legend(frameon=False)


def distribution_graph():

    ## data handling

    data = graph_data.loc[
        graph_data["small_airport"] == "S", "frequency_mhz"
    ]  # copies global dataframe and filters only small airport data

    ## graph drawing

    visualisation_left.clf()  # clears existing figures
    graph_plot = visualisation_left.add_subplot(111)  # creates graph subplot

    bins = [*range(105, 145, 1)]  # generate static bins for all histograms
    graph_plot.hist(data, bins, color="#9ad718", edgecolor="black")  # creates the graph

    distribution_canvas.draw()  # redraws the canvas to display changes
    distribution_canvas.get_tk_widget().grid()

    ## graph design

    graph_plot.set_title("Small Airport Frequency Distribution\n")  # sets the title
    graph_plot.set_xlabel("MHz")  # sets the x axis label
    graph_plot.set_ylabel("Usage")  # sets the y axis label
    graph_plot.set_axisbelow(True)  # puts grid behind drawn bars
    graph_plot.grid(True, alpha=0.2, ls="-")  # adds a faded grid
    graph_plot.margins(x=0, y=0.05663)
    graph_plot.set_ylim(0, 40)  # standardises the vertical scale across graphs


def correlation_graph():

    ## data handling

    small = graph_data.loc[graph_data["small_airport"] == "S", "frequency_mhz"]
    medium = graph_data.loc[graph_data["medium_airport"] == "M", "frequency_mhz"]
    large = graph_data.loc[graph_data["large_airport"] == "L", "frequency_mhz"]
    graph_types = [small, medium, large]

    ## graph drawing

    visualisation_right.clf()  # clears existing figures
    graph_plot = visualisation_right.add_subplot(111)  # creates graph subplot

    boxplot = graph_plot.boxplot(graph_types, patch_artist=True, showfliers=False)

    correlation_canvas.draw()  # redraws the canvas to display changes
    correlation_canvas.get_tk_widget().grid()

    ## graph design

    graph_plot.set_title("Frequency and Airport Size Correlation\n")  # sets the title
    graph_plot.set_xlabel("Size")  # sets the x axis label
    graph_plot.set_ylabel("MHz")  # sets the y axis label
    graph_plot.set_xticklabels(["Small", "Medium", "Large"])
    graph_plot.set_axisbelow(True)  # puts grid behind drawn bars
    graph_plot.grid(True, alpha=0.2, ls="-")  # adds a faded grid
    graph_plot.margins(x=0, y=0.05663)

    colors = ["#E1F0FC", "#87C3F4", "#2D95EC"]
    for patch, color in zip(boxplot["boxes"], colors):
        patch.set_facecolor(color)  # sets colours for boxes

    for median in boxplot["medians"]:
        median.set(color="#f6ba2a", linewidth=2)  # sets median band colour

Universal Helper Functions

In [6]:
# FEEDBACK


def set_feedback(status, message):  # updates feedback banner at bottom of window

    ## feedback

    if status == "error":
        feedback_label.config(text=message, background="#e56b6f")  # red error bg
    elif status == "success":
        feedback_label.config(text=message, background="#90be6d")  # green error bg

    feedback_label.after(  # resets the error or success message after set time
        4000, lambda: feedback_label.config(text="", background="#c2c2c2")
    )

# GUI - tkinter

In [8]:
# ROOT - base window layer

root = Tk()
root.title("Airport Frequency Analyser")  # sets window title
root.resizable(False, False)  # prevents window resizing to maintain intended spacing


# STYLE - set for whole program

style = ttk.Style()
style.theme_use("clam")  # sets style, used primarily for treeview column headers


# NOTEBOOK - tabs for organisation

main_notebook = ttk.Notebook(root)  # creates a notebook for subsequent tabs
main_notebook.pack(pady=(10, 0), padx=10)

data_tab = Frame(main_notebook)  # creates data tab
data_tab.pack(fill="both", expand=1)
main_notebook.add(data_tab, text="Data")  # adds tab label

averages_tab = Frame(main_notebook)  # creates averages tab
averages_tab.pack(fill="both", expand=1)
main_notebook.add(averages_tab, text="Averages")  # adds tab label

visualisations_tab = Frame(main_notebook)  # creates visualisations tab
visualisations_tab.pack(fill="both", expand=1)
main_notebook.add(visualisations_tab, text="Visualisations")  # adds tab label


# LABELFRAMES - grooved boxes for organisation

## data tab

data_lframe = LabelFrame(data_tab, text="Raw Data")  # top data window
data_lframe.pack(fill=BOTH, expand="yes", padx=10, pady=(10, 0))

file_lframe = LabelFrame(data_tab, text="Files")  # bottom left buttons
file_lframe.pack(side=LEFT, fill="x", expand="yes", padx=10, pady=10)

operations_lframe = LabelFrame(data_tab, text="Operations")  # bottom right buttons
operations_lframe.pack(side=RIGHT, fill="x", expand="yes", padx=10, pady=10)

## averages tab

averages_lframe = LabelFrame(averages_tab, text="Airport")  # both graphs
averages_lframe.pack(side=LEFT, fill=BOTH, expand="yes", padx=10, pady=10)

frequency_lframe = LabelFrame(averages_tab, text="Frequency")  # both graphs
frequency_lframe.pack(side=RIGHT, fill=BOTH, expand="yes", padx=10, pady=10)

## visualisations tab

distribution_lframe = LabelFrame(visualisations_tab, text="Distribution")  # left graph
distribution_lframe.pack(side=LEFT, fill=BOTH, expand="yes", padx=10, pady=10)

correlation_lframe = LabelFrame(visualisations_tab, text="Correlation")  # right graph
correlation_lframe.pack(side=RIGHT, fill=BOTH, expand="yes", padx=10, pady=10)


# TREEVIEW - scrollable window for displaying data

## display window

rows_frame = Frame(data_lframe)  # creates frame for scrollable data window
rows_frame.pack(padx=10, pady=10, fill="both", expand=1)

## scroll bar

tree_scroll = Scrollbar(rows_frame, orient="vertical")  # creates scroll bar
tree_scroll.pack(side=RIGHT, fill=Y)
data_tree = ttk.Treeview(rows_frame, yscrollcommand=tree_scroll.set)  # sets behaviour
tree_scroll.config(command=data_tree.yview)  #  links scrollbar to behaviour
data_tree.pack(fill="both", expand=1)

## columns

column_headers = [  # list of column names for display in treeview
    "Ref",
    "Ident",
    "Name",
    "Country",
    "Small Airport",
    "Medium Airport",
    "Large Airport",
    "Frequency Type",
    "Frequency MHz",
]

data_tree["columns"] = column_headers  # creates columns from list

data_tree.column("#0", width=0, stretch=NO)  # column widths: first off screen
data_tree.column(column_headers[0], width=100)
data_tree.column(column_headers[1], width=110)
data_tree.column(column_headers[2], width=320)
data_tree.column(column_headers[3], width=110)
data_tree.column(column_headers[4], width=110)
data_tree.column(column_headers[5], width=110)
data_tree.column(column_headers[6], width=110)
data_tree.column(column_headers[7], width=110)
data_tree.column(column_headers[8], width=110)

data_tree.heading("#0", text="", anchor=W)  # column names: first off screen
data_tree.heading(column_headers[0], text=column_headers[0], anchor=W)
data_tree.heading(column_headers[1], text=column_headers[1], anchor=W)
data_tree.heading(column_headers[2], text=column_headers[2], anchor=W)
data_tree.heading(column_headers[3], text=column_headers[3], anchor=W)
data_tree.heading(column_headers[4], text=column_headers[4], anchor=W)
data_tree.heading(column_headers[5], text=column_headers[5], anchor=W)
data_tree.heading(column_headers[6], text=column_headers[6], anchor=W)
data_tree.heading(column_headers[7], text=column_headers[7], anchor=W)
data_tree.heading(column_headers[8], text=column_headers[8], anchor=W)


# BUTTONS - clickable objects for functionality

## file labelframe (left)

airport_button = Button(  # buttons on data tab / open files label frame
    file_lframe,
    text="Open Airport Data",
    width=20,
    height=2,
    command=lambda: csv_open(airport_label),  # file open box
)
airport_button.grid(row=0, column=0, padx=10, pady=10)

frequency_button = Button(
    file_lframe,
    text="Open Frequency Data",
    width=20,
    height=2,
    command=lambda: csv_open(frequency_label),  # file open box
)
frequency_button.grid(row=0, column=1, padx=10, pady=10)

runway_button = Button(
    file_lframe,
    text="Open Runway Data",
    width=20,
    height=2,
    command=lambda: csv_open(runway_label),  # file open box
)
runway_button.grid(row=0, column=2, padx=10, pady=10)

## operations labelframe (right)

clean_button = Button(
    operations_lframe,
    text="Clean Data",
    width=20,
    height=2,
    command=clean_data,  # cleans loaded csv data
    state=DISABLED,
)
clean_button.grid(row=0, column=0, padx=10, pady=10)

merge_button = Button(
    operations_lframe,
    text="Merge Data",
    width=20,
    height=2,
    command=merge_data,  # merges loaded csv data
    state=DISABLED,
)
merge_button.grid(row=0, column=1, padx=10, pady=10)

backup_button = Button(
    operations_lframe,
    text="Save Backup",
    width=20,
    height=2,
    command=backup_json,  # saves merged data to JSON file
    state=DISABLED,
)
backup_button.grid(row=0, column=2, padx=10, pady=10)

restore_button = Button(
    operations_lframe,
    text="Load Backup",
    width=20,
    height=2,
    command=restore_json,  # loads merged data from JSON file
)
restore_button.grid(row=0, column=3, padx=10, pady=10)


# LABELS - text areas for user feedback

## file labelframe (left)

airport_label = Label(file_lframe, text="No airport file", foreground="red")
airport_label.grid(row=1, column=0, pady=10)  # displays airport file status

frequency_label = Label(file_lframe, text="No frequency file", foreground="red")
frequency_label.grid(row=1, column=1, pady=10)  # displays frequency file status

runway_label = Label(file_lframe, text="No runway file", foreground="red")
runway_label.grid(row=1, column=2, pady=10)  # displays runway file status

## operations labelframe (right)

cleaned_label = Label(operations_lframe, text=" ")
cleaned_label.grid(row=1, column=0, pady=10)  # displays cleaned file status

merged_label = Label(operations_lframe, text=" ")
merged_label.grid(row=1, column=1, pady=10)  # displays merged file status

saved_label = Label(operations_lframe, text=" ")
saved_label.grid(row=1, column=2, pady=10)  # displays saved file status

restore_label = Label(operations_lframe, text=" ")
restore_label.grid(row=1, column=3, pady=10)  # displays restored file status

## window wide feedback bar

feedback_label = Label(root, text=" ", background="#c2c2c2")  # for user feedback
feedback_label.pack(pady=10, padx=10, fill="both", expand=1)


# FIGURES - drawing area for graphs

## averages tab

averages_left = Figure(figsize=(5.7, 4.8), dpi=100)  # sets height and width
averages_left.set_facecolor("#f0f0f0")  # sets color to match window

averages_right = Figure(figsize=(5.7, 4.8), dpi=100)  # sets height and width
averages_right.set_facecolor("#f0f0f0")  # sets color to match window

## visualisations tab

visualisation_left = Figure(figsize=(5.7, 4.8), dpi=100)  # sets height and width
visualisation_left.set_facecolor("#f0f0f0")  # sets color to match window

visualisation_right = Figure(figsize=(5.7, 4.8), dpi=100)  # sets height and width
visualisation_right.set_facecolor("#f0f0f0")  # sets color to match window


# CANVAS - area for holding figures

## averages tab

airport_canvas = FigureCanvasTkAgg(  # creates airport graph canvas
    averages_left, averages_lframe
)
airport_canvas.get_tk_widget().grid(row=0, column=0, columnspan=1, pady=10, padx=10)

frequency_canvas = FigureCanvasTkAgg(  # creates freq graph canvas
    averages_right, frequency_lframe
)
frequency_canvas.get_tk_widget().grid(row=0, column=1, columnspan=3, pady=10, padx=10)

## visualisations tab

distribution_canvas = FigureCanvasTkAgg(  # creates distribution graph canvas
    visualisation_left, distribution_lframe
)
distribution_canvas.get_tk_widget().grid(
    row=0, column=0, columnspan=3, pady=10, padx=10
)

correlation_canvas = FigureCanvasTkAgg(  # creates correlation graph canvas
    visualisation_right, correlation_lframe
)
correlation_canvas.get_tk_widget().grid(row=0, column=0, columnspan=3, pady=10, padx=10)


# GUI LOOP - to start the GUI


root.mainloop()  # starts main GUI loop